<a href="https://colab.research.google.com/github/blhprasanna99/speech_emotion_detection/blob/master/speechemotion_ml_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install soundfile

In [0]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split

# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy"
}

In [0]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [0]:
def load_data(test_size=0.2):
    X, y = [], []
    try :
      for file in glob.glob("/content/drive/My Drive/wav/Actor_*/*.wav"):
          # get the base name of the audio file
          basename = os.path.basename(file)
          print(basename)
          # get the emotion label
          emotion = int2emotion[basename.split("-")[2]]
          # we allow only AVAILABLE_EMOTIONS we set
          if emotion not in AVAILABLE_EMOTIONS:
              continue
          # extract speech features
          features = extract_feature(file, mfcc=True, chroma=True, mel=True)
          # add to data
          X.append(features)
          y.append(emotion)
    except :
         pass
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=7)

In [0]:
X_train, X_test, y_train, y_test = load_data(test_size=0.25)
# print some details
# number of samples in training data
print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])
# number of features used
# this is a vector of features extracted 
# using utils.extract_features() method
print("[+] Number of features:", X_train.shape[1])

03-01-01-01-01-01-21.wav
03-01-01-01-02-02-21.wav
03-01-01-01-01-02-21.wav
03-01-02-01-01-01-21.wav
03-01-02-01-01-02-21.wav
03-01-02-01-02-01-21.wav
03-01-01-01-02-01-21.wav
03-01-02-01-02-02-21.wav
03-01-02-02-01-01-21.wav
03-01-02-02-02-01-21.wav
03-01-02-02-02-02-21.wav
03-01-02-02-01-02-21.wav
03-01-03-01-02-02-21.wav
03-01-03-01-01-01-21.wav
03-01-03-01-02-01-21.wav
03-01-03-01-01-02-21.wav
03-01-03-02-02-02-21.wav
03-01-03-02-01-01-21.wav
03-01-03-02-02-01-21.wav
03-01-03-02-01-02-21.wav
03-01-04-01-01-01-21.wav
03-01-04-01-01-02-21.wav
03-01-04-01-02-01-21.wav
03-01-04-01-02-02-21.wav
03-01-05-01-01-02-21.wav
03-01-04-02-01-01-21.wav
03-01-04-02-01-02-21.wav
03-01-05-01-02-01-21.wav
03-01-04-02-02-02-21.wav
03-01-05-01-01-01-21.wav
03-01-05-02-01-01-21.wav
03-01-05-01-02-02-21.wav
03-01-04-02-02-01-21.wav
03-01-06-01-02-02-21.wav
03-01-05-02-02-01-21.wav
03-01-06-02-01-01-21.wav
03-01-05-02-01-02-21.wav
03-01-06-02-01-02-21.wav
03-01-06-02-02-01-21.wav
03-01-06-01-01-01-21.wav


**DECISION TREE**

In [0]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 

print(accuracy_score(y_true=y_test,y_pred=dtree_predictions))
print(classification_report(y_test,dtree_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, dtree_predictions) )

0.5501519756838906
              precision    recall  f1-score   support

       angry       0.88      0.47      0.61        90
       happy       0.49      0.66      0.56        94
     neutral       0.28      0.27      0.28        44
         sad       0.59      0.64      0.61       101

    accuracy                           0.55       329
   macro avg       0.56      0.51      0.51       329
weighted avg       0.60      0.55      0.55       329

[[42 34 11  3]
 [ 6 62  9 17]
 [ 0  6 12 26]
 [ 0 25 11 65]]


In [0]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


dtree_model = DecisionTreeClassifier(max_depth = 6).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 

print(accuracy_score(y_true=y_test,y_pred=dtree_predictions))
print(classification_report(y_test,dtree_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, dtree_predictions) )

0.6595744680851063
              precision    recall  f1-score   support

       angry       0.74      0.74      0.74        90
       happy       0.57      0.53      0.55        94
     neutral       0.58      0.68      0.62        44
         sad       0.71      0.69      0.70       101

    accuracy                           0.66       329
   macro avg       0.65      0.66      0.65       329
weighted avg       0.66      0.66      0.66       329

[[67 21  0  2]
 [17 50  8 19]
 [ 4  3 30  7]
 [ 3 14 14 70]]


DT

In [0]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


dtree_model = DecisionTreeClassifier(max_depth = 9,random_state=0).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 

print(accuracy_score(y_true=y_test,y_pred=dtree_predictions))
print(classification_report(y_test,dtree_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, dtree_predictions) )

0.6656534954407295
              precision    recall  f1-score   support

       angry       0.66      0.76      0.70        90
       happy       0.63      0.55      0.59        94
     neutral       0.67      0.64      0.65        44
         sad       0.70      0.70      0.70       101

    accuracy                           0.67       329
   macro avg       0.66      0.66      0.66       329
weighted avg       0.66      0.67      0.66       329

[[68 16  0  6]
 [22 52  3 17]
 [ 4  4 28  8]
 [ 9 10 11 71]]


**SUPPORT VECTOR MACHINE**

In [0]:
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
svm_predictions = svm_model_linear.predict(X_test) 


print(accuracy_score(y_true=y_test,y_pred=svm_predictions))
print(classification_report(y_test,svm_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, svm_predictions) )

0.7507598784194529
              precision    recall  f1-score   support

       angry       0.78      0.79      0.78        90
       happy       0.73      0.70      0.71        94
     neutral       0.65      0.73      0.69        44
         sad       0.80      0.77      0.78       101

    accuracy                           0.75       329
   macro avg       0.74      0.75      0.74       329
weighted avg       0.75      0.75      0.75       329

[[71 13  4  2]
 [12 66  4 12]
 [ 3  3 32  6]
 [ 5  9  9 78]]


In [0]:
from sklearn.svm import SVC 
svm_model_linear = SVC().fit(X_train, y_train) 
svm_predictions = svm_model_linear.predict(X_test) 


print(accuracy_score(y_true=y_test,y_pred=svm_predictions))
print(classification_report(y_test,svm_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, svm_predictions) )

0.48632218844984804
              precision    recall  f1-score   support

       angry       0.69      0.66      0.67        90
       happy       0.36      0.40      0.38        94
     neutral       0.00      0.00      0.00        44
         sad       0.46      0.62      0.53       101

    accuracy                           0.49       329
   macro avg       0.38      0.42      0.39       329
weighted avg       0.43      0.49      0.45       329

[[59 27  0  4]
 [24 38  0 32]
 [ 1  5  0 38]
 [ 2 36  0 63]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**RANDOM FOREST**

In [0]:
from sklearn.ensemble import RandomForestClassifier
  
 # create regressor object 
classifier = RandomForestClassifier(n_estimators = 100, random_state = 0) 
  
# fit the regressor with x and y data 
classifier.fit(X_train, y_train)   

c_p = classifier.predict(X_test) 


print(accuracy_score(y_true=y_test,y_pred=c_p))
print(classification_report(y_test,c_p)) 
# creating a confusion matrix 
print(confusion_matrix(y_test,c_p) )

0.7659574468085106
              precision    recall  f1-score   support

       angry       0.87      0.87      0.87        90
       happy       0.70      0.72      0.71        94
     neutral       0.88      0.64      0.74        44
         sad       0.71      0.77      0.74       101

    accuracy                           0.77       329
   macro avg       0.79      0.75      0.76       329
weighted avg       0.77      0.77      0.77       329

[[78 10  0  2]
 [ 9 68  1 16]
 [ 0  2 28 14]
 [ 3 17  3 78]]


In [0]:
from sklearn.ensemble import RandomForestClassifier
  
 # create regressor object 
classifier = RandomForestClassifier(n_estimators = 20,random_state = 0) 
  
# fit the regressor with x and y data 
classifier.fit(X_train, y_train)   

c_p = classifier.predict(X_test) 


print(accuracy_score(y_true=y_test,y_pred=c_p))
print(classification_report(y_test,c_p)) 
# creating a confusion matrix 
print(confusion_matrix(y_test,c_p) )

0.7142857142857143
              precision    recall  f1-score   support

       angry       0.79      0.86      0.82        90
       happy       0.68      0.61      0.64        94
     neutral       0.72      0.59      0.65        44
         sad       0.68      0.74      0.71       101

    accuracy                           0.71       329
   macro avg       0.72      0.70      0.70       329
weighted avg       0.71      0.71      0.71       329

[[77 10  0  3]
 [14 57  4 19]
 [ 0  4 26 14]
 [ 7 13  6 75]]
